# Práctico: Introducción a Modelos de Lenguaje usando Hugging Face y GPT-2

In [1]:
# Primero, instala la librería transformers si no está instalada
# Descomenta la línea de abajo si necesitas instalar la librería
!pip install transformers

In [2]:
# Importando las librerías necesarias
import time
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import logging as transformers_logging

# Configurar el logging para que no muestre warnings de transformers
transformers_logging.set_verbosity_error()

## 1. Introducción a los Modelos de Lenguaje
------------------------------------------
Un modelo de lenguaje predice la siguiente palabra en una secuencia de texto basándose en las palabras previas.

En este práctico, utilizaremos un modelo preentrenado GPT-2, específicamente una versión distilada, que es más pequeña y rápida.

## 2. La Importancia de los Tokenizers
-----------------------------------
Los tokenizers convierten el texto en tokens numéricos, que el modelo puede entender.
Utilizaremos el tokenizer de GPT-2 para codificar texto en tokens y decodificar los tokens de vuelta a texto.

In [3]:
# Cargando el tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
# Ejemplo: Tokenizando una oración
sentence = "A long time ago in a galaxy far, far away there was an astrophysicist"

In [5]:
# Tokenizar la oración
tokens = tokenizer.encode(sentence)
print("Oración tokenizada:", tokens)

Oración tokenizada: [32, 890, 640, 2084, 287, 257, 16161, 1290, 11, 1290, 1497, 612, 373, 281, 48782, 893, 48187]


Los tokens son los números que representan las palabras o partes de las palabras.

In [6]:
# Decodificando los tokens de nuevo a texto
decoded_sentence = tokenizer.decode(tokens)
print("Oración decodificada:", decoded_sentence)

Oración decodificada: A long time ago in a galaxy far, far away there was an astrophysicist


### Tokenizador de GPT-2 vs Tokenizador BERT.

In [7]:
sentence = "a longtime ago in a galaxy far, far away there was an astrophysicist"

In [8]:
sentence = "a long time ago in a galaxy far, far away there was an astrophysicist"

In [9]:
# Cargar el tokenizer de BERT
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

print("BERT tokenizer:")

# Tokenizar con BERT
bert_tokens = bert_tokenizer.encode(sentence)
print("- Tokens:", bert_tokens)
print("- Cantidad de tokens:", len(bert_tokens))

# Tokens individuales
indivitual_tokens = bert_tokenizer.convert_ids_to_tokens(bert_tokens)
print("- Tokens individuales: ", indivitual_tokens)

# Decodificar los tokens de BERT
bert_decoded_sentence = bert_tokenizer.decode(bert_tokens)
print("- Oración decodificada:", bert_decoded_sentence, '\n')


print("GPT-2 tokenizer:")
# Comparar con el tokenizer de GPT-2
gpt2_tokens = tokenizer.encode(sentence)
print("- Tokens:", gpt2_tokens)
print("- Cantidad de tokens:", len(gpt2_tokens))

# Tokens individuales
gpt2_indivitual_tokens = tokenizer.convert_ids_to_tokens(gpt2_tokens)
print("- Tokens individuales: ", gpt2_indivitual_tokens)

# Decodificar los tokens de GPT-2
gpt2_decoded_sentence = bert_tokenizer.decode(gpt2_tokens)
print("- Oración decodificada:", gpt2_decoded_sentence)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

BERT tokenizer:
- Tokens: [101, 1037, 2146, 2051, 3283, 1999, 1037, 9088, 2521, 1010, 2521, 2185, 2045, 2001, 2019, 28625, 21281, 19570, 2923, 102]
- Cantidad de tokens: 20
- Tokens individuales:  ['[CLS]', 'a', 'long', 'time', 'ago', 'in', 'a', 'galaxy', 'far', ',', 'far', 'away', 'there', 'was', 'an', 'astro', '##phy', '##sic', '##ist', '[SEP]']
- Oración decodificada: [CLS] a long time ago in a galaxy far, far away there was an astrophysicist [SEP] 

GPT-2 tokenizer:
- Tokens: [64, 890, 640, 2084, 287, 257, 16161, 1290, 11, 1290, 1497, 612, 373, 281, 48782, 893, 48187]
- Cantidad de tokens: 17
- Tokens individuales:  ['a', 'Ġlong', 'Ġtime', 'Ġago', 'Ġin', 'Ġa', 'Ġgalaxy', 'Ġfar', ',', 'Ġfar', 'Ġaway', 'Ġthere', 'Ġwas', 'Ġan', 'Ġastroph', 'ys', 'icist']
- Oración decodificada: [unused63] [unused885] [unused635] than [unused282] [unused252]uto ـ [unused10] ـ ᵇ [unused607] [unused368] [unused276] [unused888]


### Ejercicio:
- ¿Qué son los tokens `[CLS]` y `[SEP]` que aparecen en la *Oracion decodificada* de BERT?
- ¿Por qué hay tokens que comienzan con `##` cuando usamos el tokenizador de BERT? ¿Qué significan? ¿Y los que no lo tienen?
- ¿Por qué hay tokens que comienzan con `Ġ` cuando usamos el tokenizador de distill GPT-2? ¿Qué significan? ¿Y los que no lo tienen?
- ¿Cree que son necesarios estos tokens? ¿Por qué?
- ¿Qué pasaría si usamos el tokenizador de BERT para generar texto usando distil GPT-2?

### Respuestas:
- Los tokens `[CLS]` y `[SEP]` que aparecen BERT son tokens especiales para estructurar las secuencias. En el caso de `[CLS]` se utiliza al inicio de las mismas y es un token de clasificación. Mientras que `[SEP]` marca el final de cada segmento, va al final y los delimita.
- "##" significa que el resto del token debe ser concatenado al anterior sin espacios (para decodificacion o reverso de tokenizacion). El vocabulario de BERT esta limitado, por lo que no se pueden agregar todas las palabras, divide las palabras largas o raras en partes comunes y reusa las "sub palabras".
- La `Ġ` en de distill GPT-2 significa que hay un espacio antes del token. Por ejemplo si tokenizamos la palabra "longtime" mal escrita en lugar de "long time" obtenemos 'Ġlongtime' en lugar de 'Ġlong' y 'Ġtime' como obteniamos antes
- Son necesarios porque los modelos que se entrenaron usando estos tokenizadores aprendieron el lenguaje respetando esas estructuras y reglas. Si no se utilizan esos tokens durante la inferencia, la entrada deja de coincidir con la distribución del entrenamiento y el rendimiento del modelo disminuye.
- A la hora de decodificar la frase probablemente sera totalmente distinta y hayan tokens que no sean utilizados directamente por el GPT-2. En el ejemplo anterior, al decodificar con distil GPT-2 los tokens de BERT obtenemos la siguiente frase:

`[unused63] [unused885] [unused635] than [unused282] [unused252]uto ـ [unused10] ـ ᵇ [unused607] [unused368] [unused276] [unused888]`

## 3. Cargando el Modelo Preentrenado y el Tokenizer
-------------------------------------------------
Hugging Face proporciona una gran variedad de modelos preentrenados. Usaremos el modelo 'distilgpt2' para este ejercicio.

In [10]:
# Cargar el modelo
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
# Poner el modelo en modo de evaluación
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## 4. Salida Cruda (Logits)
--------------------------------------------
Obtendremos la salida cruda del modelo (logits).

Los logits son la salida del modelo antes de la capa de activación para cada token en el vocabulario.

In [12]:
# Ejemplo de prompt
prompt = "Yesterday, I dreamed of a world where"

In [13]:
# Tokenizar la entrada
inputs = tokenizer(prompt, return_tensors="pt")

In [14]:
inputs["input_ids"].shape

torch.Size([1, 8])

In [15]:
# Pasar la entrada por el modelo para obtener los logits (puntajes sin normalizar)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Salida cruda del modelo

In [16]:
logits

tensor([[[-34.2138, -32.5889, -34.5217,  ..., -45.8648, -45.4755, -34.5368],
         [-73.7791, -73.7084, -73.1784,  ..., -77.2609, -76.5639, -71.1658],
         [-80.8371, -80.4167, -82.7387,  ..., -88.8969, -81.7753, -82.5226],
         ...,
         [-73.3639, -73.6931, -76.6878,  ..., -82.1660, -78.2536, -73.7446],
         [-56.6977, -59.0597, -63.5706,  ..., -69.0110, -65.6016, -60.3717],
         [-59.4074, -60.2301, -63.9869,  ..., -63.8053, -65.4991, -62.3250]]])

In [17]:
logits.shape

torch.Size([1, 8, 50257])

In [18]:
next_token_logits = logits[:, -1, :]  # Logits para el último token en el prompt

Aplicamos la activación SOFTMAX a `next_token_logits`

In [19]:
next_token_probs = F.softmax(next_token_logits, dim=-1)

In [20]:
print(f"Al ser probabilidades, la suma de los valores del tensor activado es {torch.sum(next_token_probs):.2}")

Al ser probabilidades, la suma de los valores del tensor activado es 1.0


In [21]:
next_token_probs

tensor([[5.2146e-05, 2.2905e-05, 5.3498e-07,  ..., 6.4151e-07, 1.1793e-07,
         2.8192e-06]])

In [22]:
next_token = torch.argmax(next_token_probs, dim=-1)
next_token

tensor([314])

In [23]:
next_word = tokenizer.decode(next_token)
next_word

' I'

### **Ejercicio:**

- Explique las dimensiones de los logits.
- ¿Qué función de activación usaría?
- Calcule la salida del modelo del siguiente token con la función de activación que respodió en la pregunta anterior.
- ¿Qué token elegiría y por qué?

### **Ejercicio:**

- Las dimensiones de los logits `torch.Size([1, 8, 50257])` tienen los siguientes significados:


  * 1 son los elementos del bache
  * 8 es la longitud en tokens de la entrada
  * 50257 es el vocabulario total del tokenizador, por lo que la salida devuelve para cada uno de los tokens de entrada + lo que viene de lo anterior, un logit por cada elemento del vocabulario que, tras softmax, da la probabilidad del siguiente token en esa posición.
- Como comentamos en la pregunta anterior, utilizariamos softmax para transformar los logits a probabilidades
- Calculamos la salida luego de aplicar la softmax:

```
tensor([[5.2146e-05, 2.2905e-05, 5.3498e-07,  ..., 6.4151e-07, 1.1793e-07,
         2.8192e-06]])
```


- Se aplica argmax para obtener el token que obtuvo una mayor probabilidad de ser el próximo, y el mismo fue el 314 que es la representación de '  I' en el vocabulario




## 5. Selección Manual de Tokens: Top-K y Top-P Sampling
-----------------------------------------------------

## Sampling con Top-K:
En Top-K sampling, seleccionamos los K tokens más probables y descartamos el resto.

Esto asegura que el modelo considere solo un número limitado de tokens de alta probabilidad, lo que ayuda a evitar tokens de baja probabilidad.

In [24]:
# Función para aplicar Top-K sampling
def top_k_sampling(logits, k=50):
    # Mantener solo los K tokens con la mayor probabilidad
    # Samplear con los tokens restantes
    # Devolver el token sampleado
    top_k_values, top_k_indices = torch.topk(logits, k)
    probs = torch.softmax(top_k_values, dim=-1)
    indice_sampleado = torch.multinomial(probs, num_samples=1)
    token_sampleado = top_k_indices[indice_sampleado]

    return token_sampleado

### Sampling con Top-P (Nucleus Sampling):
En Top-P sampling, elegimos el conjunto más pequeño de tokens cuya probabilidad acumulada supera un umbral P.

Esto significa que el modelo considera solo la porción más probable de la distribución de probabilidad.

In [74]:
# Función para aplicar Top-P (Nucleus) sampling
def top_p_sampling(logits, p=0.9):
    # Calcular la probabilidad acumulada de los tokens ordenados
    # Remover tokens cuya probabilidad acumulada sobrepasa el umbral p
    # Samplear usando los tokens restantes
    # Mapear el índice de vuelta al espacio original
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    sorted_probs = F.softmax(sorted_logits, dim=-1)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    keep_mask = cumulative_probs <= p
    keep_mask[..., 0] = True

    # 5) filtrar (ponemos -inf a lo que descartamos) y volvemos a aplicar softmax
    filtered_logits = sorted_logits.masked_fill(~keep_mask, float("-inf"))
    #print(filtered_logits.shape())
    filtered_probs = F.softmax(filtered_logits, dim=-1)

    next_idx_in_sorted = torch.multinomial(filtered_probs, num_samples=1)

    next_token_index = sorted_indices.gather(-1, next_idx_in_sorted).item()



    return next_token_index


## 6. Selección Manual de Tokens y Generación de Texto
---------------------------------------------------
Ahora usaremos los métodos de Top-K y Top-P para seleccionar manualmente el siguiente token y un texto.

In [107]:
# Función para generación manual basada en logits
def manual_text_generation(model, tokenizer, prompt, max_length=50, sampling_strategy=top_k_sampling, top_k=50, top_p=0.9):
    """
    1) Codifique el prompt inicial
    2) Configure un loop para generar tokens hasta alcanzar max_length o encontrar un token de fin de secuencia
    3) En cada iteración del loop:
        3.1) Obtenga los logits del modelo para el input
        3.2) Samplear el siguiente token usando la estrategia de muestreo elegida (Top-K o Top-P)
        3.3) Decodificar los tokens generado
        3.4) Añadir el token predicho a la secuencia de entrada y continuar generando texto
    4) Devuelva el texto generado decodificado
    """
    tokenized_prompt = tokenizer(prompt, return_tensors="pt")
    print(tokenized_prompt)

    model.eval()

    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(**tokenized_prompt)
            logits = outputs.logits
            next_token_logits = logits[:, -1, :]
            if sampling_strategy==top_k_sampling:
              next_token = sampling_strategy(next_token_logits[0], top_k)
            elif sampling_strategy == top_p_sampling:
              next_token = sampling_strategy(next_token_logits[0], top_p)
            else:
                # tomamos el proximo token con argmax
                next_token = torch.argmax(next_token_logits, dim=-1)



            # Normalizamos a int
            if isinstance(next_token, torch.Tensor):
                sampled_id = int(next_token.view(-1)[0].item())
            else:
                sampled_id = int(next_token)

            next_token_tensor = torch.tensor([[sampled_id]], dtype=torch.long)
            tokenized_prompt["input_ids"] = torch.cat([tokenized_prompt["input_ids"], next_token_tensor], dim=-1)

            if "attention_mask" in tokenized_prompt:
              ones = torch.ones((1,1), dtype=tokenized_prompt["attention_mask"].dtype, device=tokenized_prompt["attention_mask"].device)
              tokenized_prompt["attention_mask"] = torch.cat([tokenized_prompt["attention_mask"], ones], dim=-1)

            #printeamos la frase actual
            print(f"Frase actual: {tokenizer.decode(tokenized_prompt.input_ids[0], skip_special_tokens=True)}")

            eos_id = tokenizer.eos_token_id
            if eos_id is not None and sampled_id == eos_id:
                break

    return tokenizer.decode(tokenized_prompt.input_ids[0], skip_special_tokens=True)




In [108]:
prompt = "Yesterday, I dreamed of a world where I"

In [110]:
# Probamos la funcion
manual_text_generation(model, tokenizer, prompt, max_length=15, sampling_strategy=top_k_sampling, top_k=20, top_p=0.9)

{'input_ids': tensor([[28065,    11,   314, 27947,   286,   257,   995,   810,   314]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Frase actual: Yesterday, I dreamed of a world where I could
Frase actual: Yesterday, I dreamed of a world where I could play
Frase actual: Yesterday, I dreamed of a world where I could play the
Frase actual: Yesterday, I dreamed of a world where I could play the game
Frase actual: Yesterday, I dreamed of a world where I could play the game of
Frase actual: Yesterday, I dreamed of a world where I could play the game of my
Frase actual: Yesterday, I dreamed of a world where I could play the game of my life
Frase actual: Yesterday, I dreamed of a world where I could play the game of my life,
Frase actual: Yesterday, I dreamed of a world where I could play the game of my life, play
Frase actual: Yesterday, I dreamed of a world where I could play the game of my life, play video
Frase actual: Yesterday, I dreamed of a world where I could play the g

'Yesterday, I dreamed of a world where I could play the game of my life, play video games, and watch a'

In [111]:
# Probamos la funcion
manual_text_generation(model, tokenizer, prompt, max_length=15, sampling_strategy=None, top_k=1, top_p=0.9)

{'input_ids': tensor([[28065,    11,   314, 27947,   286,   257,   995,   810,   314]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Frase actual: Yesterday, I dreamed of a world where I could
Frase actual: Yesterday, I dreamed of a world where I could be
Frase actual: Yesterday, I dreamed of a world where I could be a
Frase actual: Yesterday, I dreamed of a world where I could be a little
Frase actual: Yesterday, I dreamed of a world where I could be a little more
Frase actual: Yesterday, I dreamed of a world where I could be a little more human
Frase actual: Yesterday, I dreamed of a world where I could be a little more human.
Frase actual: Yesterday, I dreamed of a world where I could be a little more human.

Frase actual: Yesterday, I dreamed of a world where I could be a little more human.


Frase actual: Yesterday, I dreamed of a world where I could be a little more human.



Frase actual: Yesterday, I dreamed of a world where I could be a little more human.


I
Fras

'Yesterday, I dreamed of a world where I could be a little more human.\n\n\nI was born in the'

In [112]:
manual_text_generation(model, tokenizer, prompt, max_length=30, sampling_strategy=top_p_sampling, top_k=50, top_p=0.8)

{'input_ids': tensor([[28065,    11,   314, 27947,   286,   257,   995,   810,   314]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Frase actual: Yesterday, I dreamed of a world where I could
Frase actual: Yesterday, I dreamed of a world where I could use
Frase actual: Yesterday, I dreamed of a world where I could use the
Frase actual: Yesterday, I dreamed of a world where I could use the power
Frase actual: Yesterday, I dreamed of a world where I could use the power of
Frase actual: Yesterday, I dreamed of a world where I could use the power of his
Frase actual: Yesterday, I dreamed of a world where I could use the power of his hands
Frase actual: Yesterday, I dreamed of a world where I could use the power of his hands to
Frase actual: Yesterday, I dreamed of a world where I could use the power of his hands to take
Frase actual: Yesterday, I dreamed of a world where I could use the power of his hands to take care
Frase actual: Yesterday, I dreamed of a world where I coul

'Yesterday, I dreamed of a world where I could use the power of his hands to take care of my kids. I always thought of myself as a doctor and as a fellow entrepreneur.\n\n'

### Ejercicio:
- Complete el código de la función `mual_text_generation` y genere texto jugando con los hyper-parámetros.
- Compare salidas con las estrategias de sampleo `top p` y `top k`.

## 7. Experimentando con Hiperparámetros
-------------------------------------
Hugging face ya nos provee con una implementación de todos estos métodos. Simplemente tenemos que pasarle por parámetro los valores que queremos usar a la hora de generar.

Ahora, exploremos cómo podemos controlar la salida del modelo cambiando los parámetros de generación.

Intenta cambiar los valores de `max_length`, `temperature`, `top_k` y `top_p`.

In [98]:
# Tokenizar la entrada
inputs = tokenizer(prompt, return_tensors="pt")

In [94]:
# Experimentar con diferentes configuraciones
output = model.generate(
    **inputs,
    max_length=20,
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)

In [102]:
# Decodificar y mostrar el texto generado
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("\nTexto generado con parámetros modificados:\n", generated_text.replace('\n', ' '))


Texto generado con parámetros modificados:
 Yesterday, I dreamed of a world where I could be a little more human.   I


### Ejercicio:
- Compare este resultado con el generado en el punto anterior.
- ¿Cómo afecta la `temperatura` a los resultados del modelo?
- ¿Cómo afecta el `top_p` a los resultados del modelo?
- ¿Cómo afecta la `top_k` a los resultados del modelo?

## 8. Sesgo en modelos de lenguaje
-----------------------------------
Ahora, exploraremos los sesgos que pueden tener estos modelos de lenguaje.

In [32]:
# Ejercicio: Detectar sesgos en la generación de texto usando diferentes prompts
prompts = [...]

for _ in range(5):
    for prompt in prompts:
        tokens = tokenizer(prompt, return_tensors='pt')
        output = model.generate(
            **tokens,
            max_length=20,
            temperature=0.7,
            top_k=50,
            top_p=.9,
            do_sample=True,
        )
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        print("Prompt: ",prompt)
        print("Texto generado: ", generated_text.replace('\n', ' '))
        print("-" * 50)


ValueError: text input must be of type `str` (single example), `list[str]` (batch or single pretokenized example) or `list[list[str]]` (batch of pretokenized examples).

### Ejercicio:
- ¿Nota algún sesgo en las respuestas de este modelo?
- Proponga prompts que puedan detectar sesgos en este modelo.
- ¿A qué se deben estos sesgos?

## 9. Comparando con otro LM
-----------------------------------
Ahora, comprararemos a distilled gpt-2 contra gpt-2.

In [ ]:
# Cargar modelo distilado y completo para comparar
distil_gpt2_model = AutoModelForCausalLM.from_pretrained('distilgpt2')
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2')  # GPT-2 completo

gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
distil_gpt2_model.eval()
gpt2_model.eval()

In [ ]:
# Puedes ejecutar el mismo texto para ambos modelos
prompt = "Photosyntheis is a process that"
max_length = 20

# Tokenizamos el texto


print("Texto de entrada:", prompt)

# Generación con el modelo distilado
print("\nGeneración con el modelo distilado:")

inference_time_distil = ...
generated_text_distil = ...

print("- Texto generado: '", generated_text_distil, "'")
print("- Tiempo de inferencia: ", inference_time_distil)


# Generación con el modelo completo
print("\nGeneración con el modelo completo:")

inference_time_gpt2 = ...
generated_text_gpt2 = ...

print("- Texto generado por GPT-2: '", generated_text_gpt2, "'")
print("- Tiempo de inferencia: ", inference_time_gpt2)


### Ejercicio:

- Compare los dos modelos y sus resultados usando diferentes *prompts* y valores de *max_length*.
- ¿Es necesario usar el parámetro `max_legth`? ¿Por qué?

## 10. Entendiendo la Salida del Modelo
-----------------------------------
Después de generar algunos ejemplos, reflexiona sobre la calidad del texto generado:
 - ¿Considera que estos modelos generan texto coherente?
 - ¿Qué tan creativo o repetitivo es el texto?
 - ¿Hay ejemplos donde la generación falla o produce resultados sin sentido?